In [ ]:
from google.colab import drive

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import joblib


df = pd.read_csv('/content/drive/MyDrive/data/creditcard.csv')


print("Dataset shape:", df.shape)
print(df['Class'].value_counts())

Dataset shape: (284807, 31)
Class
0    284315
1       492
Name: count, dtype: int64


In [2]:
df.head(5)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
X = df.drop('Class', axis=1)
y = df['Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Handle imbalanced data using SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("After SMOTE - Class distribution:", np.bincount(y_train_smote))

#  Random Forest Classifier
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train_smote, y_train_smote)


# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Testing Interface
def test_fraud_detection():
    """
    Command-line interface for testing the fraud detection system.
    """
    print("\n--- Fraud Detection System ---")
    print("Enter transaction details as a comma-separated list (match the dataset's features):")

    try:
        # Input example: "0,0,0.1,...,22.0" (fill with sample data from your dataset)
        input_data = input("Transaction data: ")
        data_array = np.array([float(x) for x in input_data.split(",")]).reshape(1, -1)

        # Load the model
        loaded_model = joblib.load('fraud_detection_model.pkl')

        # Predict fraud
        prediction = loaded_model.predict(data_array)
        result = "Fraudulent" if prediction[0] == 1 else "Not Fraudulent"

        print(f"Prediction: {result}")
    except Exception as e:
        print("Error:", e)

# Call the testing interface
test_fraud_detection()

After SMOTE - Class distribution: [199020 199020]
Confusion Matrix:
 [[85279    16]
 [   30   118]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.88      0.80      0.84       148

    accuracy                           1.00     85443
   macro avg       0.94      0.90      0.92     85443
weighted avg       1.00      1.00      1.00     85443


--- Fraud Detection System ---
Enter transaction details as a comma-separated list (match the dataset's features):


Class 0 (Non-Fraudulent Transactions):

Precision: 1.00 – All predicted non-fraudulent transactions are correct.
Recall: 1.00 – All actual non-fraudulent transactions are correctly identified.
F1-Score: 1.00 – Perfect balance between precision and recall.
Support: 85,295 – Number of actual non-fraudulent transactions.
Class 1 (Fraudulent Transactions):

Precision: 0.88 – 88% of predicted fraudulent transactions are correct.
Recall: 0.80 – 80% of actual fraudulent transactions are correctly identified.
F1-Score: 0.84 – Weighted average of precision and recall.
Support: 148 – Number of actual fraudulent transactions.
Overall Metrics:

Accuracy: 1.00 – Model's overall correct predictions.
Macro Average: Averages precision, recall, and F1-score across classes equally, irrespective of their support.
Weighted Average: Averages metrics weighted by support, favoring the larger class (Class 0).